In [76]:
import polars as pl
import os
import json
from IPython.core.interactiveshell import InteractiveShell
from IPython.display import display_html
InteractiveShell.ast_node_interactivity = "all"
import os
import json
import pandas as pd
from typing import Dict, Any
from urllib.request import urlretrieve
import seaborn as sns
import matplotlib.pyplot as plt
import math
import numpy as np
import pandas as pd
import geopandas as gpd
import folium
from shapely.geometry import Point
from shapely import wkt
import openrouteservice
import polars as pl
import os
from IPython.core.interactiveshell import InteractiveShell
import pyproj
from pyproj import CRS
from pyproj import Transformer


domain_dir = '../data/landing/domain/'

In [77]:
def get_first_valid(data, *keys):
    '''
    Get the first valid value from a list of keys in a dictionary
    '''
    for key in keys:
        try:
            value = eval(f"data{key}")
            if value is not None:
                return value
        except (KeyError, TypeError):
            continue
    return None

In [78]:
def read_json_file(file_path):
    '''
    Read a JSON file and extract relevant information into a dictionary    
    '''
    with open(file_path, 'r') as file:
        data = json.load(file)
        agency_id = get_first_valid(data, '["props"]["rootGraphQuery"]["listingByIdV2"]["agency"]["agencyId"]')
        listing_id = get_first_valid(data, '["props"]["id"]' , '["props"]["listingId"]')
        limited_agency_mode = get_first_valid(data, '["props"]["limitedAgencyMode"]')
        promotion_level = get_first_valid(data, '["props"]["rootGraphQuery"]["listingByIdV2"]["promoLevel"]')
        photo_count = get_first_valid(data, '["digitalData"]["page"]["pageInfo"]["property"]["photoCount"]')
        property_type = get_first_valid(data, '["props"]["listingSummary"]["propertyType"]', '["props"]["propertyType"]')
        num_bedrooms = get_first_valid(data, '["props"]["beds"]', '["props"]["listingSummary"]["beds"]')
        num_bathrooms = get_first_valid(data, f'["props"]["listingsMap"]["{listing_id}"]["listingModel"]["features"]["baths"]', '["props"]["listingSummary"]["baths"]', '["props"]["rootGraphQuery"]["listingByIdV2"]["bathrooms"]')
        parking = get_first_valid(data, f'["props"]["listingsMap"]["{listing_id}"]["listingModel"]["features"]["parking"]', '["props"]["listingSummary"]["parking"]', '["props"]["rootGraphQuery"]["listingByIdV2"]["carspaces"]')
        is_rural = get_first_valid(data, f'["props"]["listingsMap"]["{listing_id}"]["listingModel"]["features"]["isRural"]') 
        price = get_first_valid(data, '["props"]["rootGraphQuery"]["listingByIdV2"]["priceDetails"]["rawValues"]["exactPriceV2"]')
        bond = get_first_valid(data, '["props"]["rootGraphQuery"]["listingByIdV2"]["priceDetails"]["rawValues"]["bond"]')
        energy_eff_rating = get_first_valid(data, '["props"]["rootGraphQuery"]["listingByIdV2"]["energyEfficiencyRating"]')        
        nbn_download = get_first_valid(data, '["digitalData"]["page"]["pageInfo"]["property"]["nbnDetails"]["downloadSpeedInMbps"]')
        nbn_upload = get_first_valid(data, '["digitalData"]["page"]["pageInfo"]["property"]["nbnDetails"]["uploadSpeedInMbps"]')
        nbn_tech = get_first_valid(data, '["digitalData"]["page"]["pageInfo"]["property"]["nbnDetails"]["techType"]')
        nbn_speed_tier = get_first_valid(data, '["digitalData"]["page"]["pageInfo"]["property"]["nbnDetails"]["speedTier"]')
        nbn_service_status = get_first_valid(data, '["digitalData"]["page"]["pageInfo"]["property"]["nbnDetails"]["serviceStatus"]')
        
        
        unit_number = get_first_valid(data, '["props"]["address"]')
        street_number = get_first_valid(data, '["props"]["streetNumber"]')
        address = get_first_valid(data, '["props"]["address"]')
        suburb = get_first_valid(data, f'["props"]["listingsMap"]["{listing_id}"]["listingModel"]["address"]["suburb"]')
        state = get_first_valid(data, f'["props"]["listingsMap"]["{listing_id}"]["listingModel"]["address"]["state"]', '["props"]["stateAbbreviation"]')
        listing_date = get_first_valid(data, '["props"]["createdOn"]', '["props"]["domainSays"]["firstListedDate"]', '["props"]["rootGraphQuery"]["listingByIdV2"]["dateCreated"]["isoDate"]', '["digitalData"]["page"]["pageInfo"]["issueDate"]')
        actual_listing_date = get_first_valid(data, '["digitalData"]["page"]["pageInfo"]["property"]["dateListed"]')
        num_days_listed = get_first_valid(data, '["digitalData"]["page"]["pageInfo"]["property"]["daysListed"]')
        inspections_count = get_first_valid(data, '["digitalData"]["page"]["pageInfo"]["property"]["inspectionsCount"]')
        last_modification_date = get_first_valid(data, '["props"]["modifiedOn"]', '["props"]["rootGraphQuery"]["listingByIdV2"]["dateUpdated"]["isoDate"]')
        # Surrounding Suburbs = ["props"]["footer"]."surroundingSuburbs"
        postcode = get_first_valid(data, '["props"]["postcode"]')
        latitude = get_first_valid(data, '["props"]["map"]["latitude"]', f'["props"]["listingsMap"]["{listing_id}"]["listingModel"]["address"]["lat"]')
        longitude = get_first_valid(data, '["props"]["map"]["longitude"]', f'["props"]["listingsMap"]["{listing_id}"]["listingModel"]["address"]["lng"]')
        # Features = Process ["props"]["structuredFeatures"] to create individual binary features.
        # School Data = Extract relevant information from ["props"]["schoolCatchment"]."schools" (e.g., distance to nearest primary school, number of schools within a certain radius).
        has_photo = get_first_valid(data, '["digitalData"]["page"]["pageInfo"]["property"]["hasPhoto"]')
        has_floorplan = get_first_valid(data, '["digitalData"]["page"]["pageInfo"]["property"]["hasFloorplan"]')
        # education_level = ["props"]["schoolCatchment"]["schools"]["educationLevel"]
        # distance_ = ["props"]["schoolCatchment"]["schools"]["distance"]
        # school_postcode = ["props"]["schoolCatchment"]."schools"."postCode"
        num_visible_schools = get_first_valid(data, '["props"]["schoolCatchment"]["numberOfVisibleSchools"]')
        
        return {
            "agency_id" : agency_id,
            "listing_id" : listing_id,
            "limited_agency_mode" : limited_agency_mode,
            "promotion_level" : promotion_level,
            "photo_count" : photo_count,
            "property_type" : property_type,
            "num_bedrooms" : num_bedrooms,
            "num_bathrooms" : num_bathrooms,
            "parking" : parking,
            "is_rural" : is_rural,
            "price" : price,
            "bond" : bond,
            "energy_eff_rating" : energy_eff_rating,
            "nbn_download" : nbn_download,
            "nbn_upload" : nbn_upload,
            "nbn_tech" : nbn_tech,
            "nbn_speed_tier" : nbn_speed_tier,
            "nbn_service_status" : nbn_service_status,           
            "unit_number" : unit_number,
            "street_number" : street_number,
            "address" : address,
            "suburb" : suburb,
            "state" : state,
            "listing_date" : listing_date,
            "actual_listing_date" : actual_listing_date,
            "num_days_listed" : num_days_listed,
            "inspections_count" : inspections_count,
            "last_modification_date" : last_modification_date,
            # Surrounding Suburbs = ["props"]["footer"]."surroundingSuburbs"
            "postcode" : postcode,
            "latitude" : latitude,
            "longitude" : longitude,
            # Features = Process ["props"]["structuredFeatures"] to create individual binary features.
            # School Data = Extract relevant information from ["props"]["schoolCatchment"]."schools" (e.g., distance to nearest primary school, number of schools within a certain radius).
            "has_photo" : has_photo,
            "has_floorplan" : has_floorplan,
            # education_level = ["props"]["schoolCatchment"]["schools"]["educationLevel"]
            # distance_ = ["props"]["schoolCatchment"]["schools"]["distance"]
            # school_postcode = ["props"]["schoolCatchment"]."schools"."postCode"
            "num_visible_schools" : num_visible_schools            
        }
    
def read_json_directory(directory_path):
    # List to hold all JSON data
    json_data_list = []

    # Iterate through all files in the directory
    for filename in os.listdir(directory_path):
        if filename.endswith('.json'):
            file_path = os.path.join(directory_path, filename)
            json_data = read_json_file(file_path)
            json_data_list.append(json_data)

    # Create a Polars DataFrame from the list of JSON data
    df = pl.DataFrame(json_data_list,infer_schema_length=17000)
    return df

In [79]:
# For example, we will take the first file in the directory
if not os.path.exists(domain_dir):
    print(f"Directory {domain_dir} does not exist")

else:
    df = read_json_directory(domain_dir)


In [80]:
df = df.to_pandas()


In [81]:
df

,agency_id,listing_id,limited_agency_mode,promotion_level,photo_count,property_type,num_bedrooms,num_bathrooms,parking,is_rural,...,actual_listing_date,num_days_listed,inspections_count,last_modification_date,postcode,latitude,longitude,has_photo,has_floorplan,num_visible_schools
0,8753,16889460,False,ELITE,10,House,3,1,2,False,...,2024-02-21T09:47:30.000,198,0,2024-02-21T09:47:28.973,3020,-37.774926,144.834817,True,False,3
1,19,17188457,False,STANDARD_PP,15,House,3,2,0,False,...,2024-09-02T14:46:14.000,3,0,2024-09-02T14:46:11.913,3058,-37.748181,144.958473,True,True,3
2,6525,17160233,False,PREMIUM_PLUS,11,House,4,2,2,False,...,2024-08-14T09:50:19.000,23,1,2024-08-14T09:50:15.757,3124,-37.842214,145.071069,True,True,3
3,37657,17062496,False,ELITE,21,House,4,2,2,False,...,2024-06-13T17:32:02.000,84,0,2024-06-13T17:31:57.95,3029,-37.835021,144.669242,True,True,3
4,26896,17179442,False,ELITE,5,House,4,2,2,False,...,2024-08-27T10:59:51.000,10,0,2024-08-27T10:59:47.397,3358,-37.567505,143.784207,True,False,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15480,17402,16644757,False,ELITE,11,Townhouse,4,3,2,False,...,2024-06-11T17:29:53.000,86,0,2024-06-11T17:29:53.153,3151,-37.856688,145.132631,True,False,3
15481,27481,16086828,False,STANDARD,18,House,4,2,2,False,...,2024-09-04T10:59:52.000,2,1,2024-09-04T10:59:51.783,3030,-37.963369,144.707632,True,False,3
15482,7876,16076685,False,ELITE,6,Apartment / Unit / Flat,3,2,2,False,...,2024-09-05T10:53:31.000,1,0,2024-09-05T10:53:30.713,3156,-37.882581,145.267643,True,False,3
15483,11919,17174571,False,ELITE,10,Townhouse,3,3,2,False,...,2024-08-22T23:21:09.000,14,0,2024-08-22T23:21:09.067,3058,-37.737269,144.982527,True,False,3


In [82]:
columns_to_drop = ["agency_id", "listing_id", "limited_agency_mode", "promotion_level", "photo_count", "bond", "energy_eff_rating", "nbn_download", "nbn_upload", "nbn_tech", "nbn_speed_tier", "nbn_service_status", "unit_number", "street_number", "listing_date", "actual_listing_date", "num_days_listed", "inspections_count", "last_modification_date", "has_photo", "has_floorplan", "num_visible_schools"]
df = df.drop(columns=[col for col in columns_to_drop if col in df.columns])


In [83]:
df = df.drop(["address","is_rural","state"], axis=1)

In [84]:

print(list(df['property_type'].unique()))

# ['townhouse', 'house', 'villa', 'studio' , 'duplex-semi-detached', 'duplex', 'serviced apartment', 'retirement living']

# how to change values of the column, hence translating the values

df['property_type'] = df['property_type'].str.replace('Semi-Detached', 'duplex-semi-detached')   
df['property_type'] = df['property_type'].str.replace('Duplex', 'duplex')
df['property_type'] = df['property_type'].str.lower()
df['property_type'] = df['property_type'].str.replace('new apartments / off the plan','apartment / unit / flat')
df['property_type'] = df['property_type'].str.replace('house','house')
df['property_type'] = df['property_type'].str.replace('townhouse','townhouse')
df['property_type'] = df['property_type'].str.replace('studio','studio')
df['property_type'] = df['property_type'].str.replace('serviced apartment','serviced apartment')
df['property_type'] = df['property_type'].str.replace('villa','villa')
df['property_type'] = df['property_type'].str.replace('retirement','retirement living')
df['property_type'] = df['property_type'].str.replace('new house & land','house')



['House', 'Studio', 'Apartment / Unit / Flat', 'Townhouse', 'Vacant land', 'Acreage / Semi-Rural', 'New House & Land', 'Terrace', 'New Apartments / Off the Plan', 'Villa', 'Carspace', 'Retirement', 'Semi-Detached', 'Rural', 'Duplex', 'Block of Units', 'New land']


In [85]:
df = df.rename(columns={'price': 'rent'})


In [86]:
df

,property_type,num_bedrooms,num_bathrooms,parking,rent,suburb,postcode,latitude,longitude
0,house,3,1,2,420.0,Sunshine North,3020,-37.774926,144.834817
1,house,3,2,0,850.0,Coburg,3058,-37.748181,144.958473
2,house,4,2,2,1300.0,Camberwell,3124,-37.842214,145.071069
3,house,4,2,2,580.0,Tarneit,3029,-37.835021,144.669242
4,house,4,2,2,440.0,Winter Valley,3358,-37.567505,143.784207
...,...,...,...,...,...,...,...,...,...
15480,townhouse,4,3,2,1050.0,Burwood East,3151,-37.856688,145.132631
15481,house,4,2,2,560.0,Werribee South,3030,-37.963369,144.707632
15482,apartment / unit / flat,3,2,2,700.0,Ferntree Gully,3156,-37.882581,145.267643
15483,townhouse,3,3,2,835.0,Coburg,3058,-37.737269,144.982527


In [87]:
df['rent'] = df['rent'].fillna(0)
df = df[df['rent'] > 0.0]
df = df[df['num_bedrooms'] > 0.0]
df = df[df['num_bathrooms'] > 0.0]
df = df[df['parking'] >= 0.0]
df = df[df['property_type'] != '']
df['suburb'] = df['suburb'].str.lower()
df['property_type'] = df['property_type'].str.lower()
df = df[df['longitude'] != 0.0]
df = df[df['latitude'] != 0.0]

In [88]:
df = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude))

df = df.set_crs(epsg=4326, allow_override=True) 


df = df.to_crs(epsg=32755) 


In [90]:
def distance_from_cbd(point):
    cbd_lat =  -37.810272
    cbd_long = 144.962646  
    crs = CRS.from_epsg(32755)



    transformer = Transformer.from_crs("epsg:4326", "epsg:32755", always_xy=True)

    utm_x, utm_y = transformer.transform(cbd_long, cbd_lat)
    pointcbd = Point(utm_x, utm_y)


    return pointcbd.distance(point)


df['distance_to_cbd'] = df['geometry'].apply(lambda x: distance_from_cbd(x))


In [92]:
df['distance_to_cbd'] = df['distance_to_cbd']/1000

In [98]:
station_df = gpd.read_file('../data/raw/PTV/PTV_METRO_TRAIN_STATION.shp')
station_df = station_df.to_crs(epsg=32755)


In [99]:
df_st_ = gpd.sjoin_nearest(df, station_df[['STOP_NAME', 'geometry']], how="left", distance_col='nearst_train_station_dist')


In [100]:
df_st_

,property_type,num_bedrooms,num_bathrooms,parking,rent,suburb,postcode,latitude,longitude,geometry,distance_to_cbd,index_right,STOP_NAME,nearst_train_station_dist
0,house,3,1,2,420.0,sunshine north,3020,-37.774926,144.834817,POINT (309311.792 5816949.235),11.923216,29,Albion Railway Station (Sunshine North),940.065236
1,house,3,2,0,850.0,coburg,3058,-37.748181,144.958473,POINT (320138.612 5820162.038),6.901368,216,Moreland Railway Station (Coburg),758.249838
2,house,4,2,2,1300.0,camberwell,3124,-37.842214,145.071069,POINT (330274.798 5809938.326),10.182372,100,Hartwell Railway Station (Camberwell),441.276045
3,house,4,2,2,580.0,tarneit,3029,-37.835021,144.669242,POINT (294893.986 5809929.749),25.979382,172,Hoppers Crossing Railway Station (Hoppers Cros...,6039.058778
4,house,4,2,2,440.0,winter valley,3358,-37.567505,143.784207,POINT (215973.314 5837306.858),107.405017,76,Diggers Rest Railway Station (Diggers Rest),82924.724553
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15480,townhouse,4,3,2,1050.0,burwood east,3151,-37.856688,145.132631,POINT (335724.226 5808442.4),15.825365,125,Mount Waverley Railway Station (Mount Waverley),2098.144186
15481,house,4,2,2,560.0,werribee south,3030,-37.963369,144.707632,POINT (298622.95 5795770.353),28.143669,171,Werribee Railway Station (Werribee),8197.645243
15482,apartment / unit / flat,3,2,2,700.0,ferntree gully,3156,-37.882581,145.267643,POINT (347655.729 5805798.253),28.016795,118,Ferntree Gully Railway Station (Ferntree Gully),2431.006816
15483,townhouse,3,3,2,835.0,coburg,3058,-37.737269,144.982527,POINT (322231.864 5821418.894),8.289849,37,Preston Railway Station (Preston),1594.189206
